# PR checks

In [1]:
import pennylane as qml

## PR 1

https://github.com/PennyLaneAI/pennylane/pull/5036

Checking docstrings:

In [2]:
from pennylane.qchem.openfermion_obs import dipole_of
from pennylane import numpy as np

symbols = ["H", "H", "H"]
coordinates = np.array([0.028, 0.054, 0.0, 0.986, 1.610, 0.0, 1.855, 0.002, 0.0])
dipole_obs = dipole_of(symbols, coordinates, charge=1)
print(dipole_obs)

[<Hamiltonian: terms=18, wires=[0, 1, 2, 3, 4, 5]>, <Hamiltonian: terms=18, wires=[0, 1, 2, 3, 4, 5]>, <Hamiltonian: terms=1, wires=[0]>]


In [3]:
print(dipole_obs[0])  # x-component of D

  (0.24190977644638015) [Z4]
+ (0.24190977644638015) [Z5]
+ (0.4781123173263871) [Z0]
+ (0.4781123173263871) [Z1]
+ (0.7144779061811476) [Z2]
+ (0.7144779061811476) [Z3]
+ (-0.39136384894889287) [Y0 Z1 Y2]
+ (-0.39136384894889287) [X0 Z1 X2]
+ (-0.39136384894889287) [Y1 Z2 Y3]
+ (-0.39136384894889287) [X1 Z2 X3]
+ (-0.11734958781015546) [Y2 Z3 Y4]
+ (-0.11734958781015546) [X2 Z3 X4]
+ (-0.11734958781015546) [Y3 Z4 Y5]
+ (-0.11734958781015546) [X3 Z4 X5]
+ (0.26611147045283673) [Y0 Z1 Z2 Z3 Y4]
+ (0.26611147045283673) [X0 Z1 Z2 Z3 X4]
+ (0.26611147045283673) [Y1 Z2 Z3 Z4 Y5]
+ (0.26611147045283673) [X1 Z2 Z3 Z4 X5]


Docstring are correctly rendered in documentation

## PR 2

https://github.com/PennyLaneAI/pennylane/pull/5083

This is simply a modification to the deprecations and removals guide. I think there is nothing to test.

I asked Matt and noticed where changes in the documentations are.

Docstring are correctly rendered in documentation

## PR 3

https://github.com/PennyLaneAI/pennylane/pull/5237

Checking docstrings:

In [4]:
dev = qml.device("default.qubit", shots=10)


@qml.qnode(dev, diff_method="parameter-shift")
def circuit(angle):
    qml.RX(angle, wires=0)
    return qml.sample(qml.PauliX(0))


angle = qml.numpy.array(0.1)
res = qml.jacobian(circuit)(angle)

In [5]:
dev = qml.device("default.qubit", shots=[(1, 10)])


@qml.qnode(dev, diff_method="parameter-shift")
def circuit(angle):
    qml.RX(angle, wires=0)
    return qml.expval(qml.PauliX(0))


def cost(angle):
    return qml.math.hstack(circuit(angle))


angle = qml.numpy.array(0.1)
res = qml.jacobian(cost)(angle)

Docstring are correctly rendered in documentation

## PR 4

https://github.com/PennyLaneAI/pennylane/pull/5080

Testing tests:

In [6]:
op = qml.CosineWindow([2, 1])
res = op.state_vector(wire_order=[0, 1, 2])
assert res.shape == (2, 2, 2)

expected_10 = qml.CosineWindow([0, 1]).state_vector(wire_order=[1, 0])
expected = np.stack([expected_10, np.zeros_like(expected_10)])
assert np.allclose(res, expected)

Everything seems fine. I could double check that the new code lines are actually executed, but I'll do that later if I have time

## PR 5

https://github.com/PennyLaneAI/pennylane/pull/5178

Checking the bug report to be sure it is fixed:

In [7]:
import tensorflow as tf


def grad(f):
    def wrapper(x):
        with tf.GradientTape() as tape:
            y = f(x)

        return tape.gradient(y, x)

    return wrapper


dev = qml.device("default.qubit")


@qml.qnode(dev)
def circuit(theta):
    meas = qml.iterative_qpe(qml.RZ(theta, wires=[0]), [1], iters=2)
    return qml.expval(meas[0])


@qml.qnode(dev)
def manual_circuit(phi):
    qml.Hadamard(wires=[1])
    qml.ctrl(qml.RZ(phi, wires=[0]) ** 2, control=[1])
    qml.Hadamard(wires=[1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 1])
    qml.Hadamard(wires=[1])
    qml.ctrl(qml.RZ(phi, wires=[0]), control=[1])
    qml.ctrl(qml.PhaseShift(-np.pi / 2, wires=[1]), control=[2])
    qml.Hadamard(wires=[1])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[3, 1])

    return qml.expval(qml.Hermitian([[0, 0], [0, 1]], wires=3))


phi = tf.Variable(1.0)
assert np.isclose(grad(circuit)(phi), grad(manual_circuit)(phi))

2024-02-26 12:57:59.492038: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 12:57:59.519751: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 12:57:59.519783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 12:57:59.520528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 12:57:59.525224: I tensorflow/core/platform/cpu_feature_guar

## PR 6

https://github.com/PennyLaneAI/pennylane/pull/5233

In [14]:
import pennylane as qml
from pennylane import numpy as pnp

import numpy as np


@qml.qnode(qml.device("default.qubit", wires=3), diff_method="adjoint")
def circuit(x):
    qml.RX(x, 0)
    qml.QubitUnitary(np.eye(8), [0, 1, 2])
    return qml.expval(qml.PauliZ(2))


x = pnp.array(1.1, requires_grad=True)
qml.jacobian(circuit)(x)

array(0.)

Solved: for this specific error there was an import error that needed to be changed: `numpy` had to be imported not from `pennylane`

## PR 7

https://github.com/PennyLaneAI/pennylane/pull/5091

This is just a changing in the doc. The modification is also present in the official documentation, so everything is fine.

## PR 8

https://github.com/PennyLaneAI/pennylane/pull/5047

Learned from Matt that I need to change to `latest` in the doc to see last modifications